In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import numpy as np
import scipy as sp
import random
import itertools
from pydash import at
import cProfile
import re
from numba import jitclass
from numba import int32, float32
from mpl_toolkits.mplot3d import Axes3D
import pickle


In [3]:
class Graph:

    def __init__(self, n_v, n_e, a=1.7, eps=0.4):
        """
        @n_v: number of nodes (int)
        @n_e: number of edges (int)
        @a: control parameter of logistic map (float)
        @eps: coupling strength (float)

        Initializes a random network.
        """

        # Save parameters
        self.n_v = n_v
        self.n_e = n_e
        self.a = a
        self.eps = eps

        # Define arrays to store CC and CLS
        self.cc = []
        self.cls = []

        # Set seed for testing
        np.random.seed(1337)

        # Initialize random network
        self.G = nx.gnm_random_graph(n_v, n_e, seed=1337)

        # Compute initial value for each node
        init_values = np.random.uniform(-1, 1, n_v)
        init_attr = dict()
#         self.init_pos1= dict()
#         self.init_pos = dict()
#         posdictinit=dict()
#         for i in range(n_v):
#             posdictinit[i]=[random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]
        for i in range(n_v):
            init_attr[i] = {'value': init_values[i]}
#         self.init_pos1 = nx.drawing.layout.kamada_kawai_layout(self.G,pos=posdictinit,dim=3)
#         self.new_pos1=self.init_pos1
#         m=0
#         for key, value in self.init_pos1.items():
#             self.init_pos[m]={'pos':value}
#             m+=1
        #print(init_pos)
            #init_pos[i] = {'pos': (random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1))}
        

        # Set initial value for each node
        #nx.set_node_attributes(self.G, self.init_pos)
        nx.set_node_attributes(self.G, init_attr)
        

    def timestep(self, t):
        """
        @t: number of timesteps to be made (int)

        Performs a number of rewiring timesteps
        """

        # Perform t iterations
        for i in range(t):

            # Update values in nodes
            self.update_attr()

            # Determine pivot, candidate and outcast
            pivot, candidate, outcast = self.pivot()

            # Rewire if possible
            self.rewire(pivot, candidate, outcast)
            
            #Update the positions 
            #self.update_pos()

            self.cc.append(nx.average_clustering(self.G))

        print('Timestep done')
        
        
#     def update_pos(self):
        
#         self.new_pos1=nx.drawing.layout.kamada_kawai_layout(self.G,pos=self.new_pos1,dim=3)
#         self.new_pos=dict()
#         m=0
#         for key,value in self.new_pos1.items():
#             self.new_pos[m]={'pos':value}
#             m+=1
        
#         nx.set_node_attributes(self.G,self.new_pos)

    def update_attr(self):
        """
        Calculates all new values of all nodes in the graph.
        """

        # Retrieve all current values
        all_values = nx.get_node_attributes(self.G, 'value')

        # Loop over all nodes
        for i in range(self.n_v):

            # Obtain list of neighbors
            neighbors = list(nx.all_neighbors(self.G, i))

            # Compute part dependent on own node
            new_value = (1 - self.eps) * self.logistic_map(all_values[i])

            # Compute part dependent on neighbor nodes
            neighbors_value = 0
            for j, neighbor in enumerate(neighbors):
                neighbors_value += self.logistic_map(all_values[neighbor])

            # Catch nodes without neighbors
            try:
                new_value += neighbors_value * (self.eps/len(neighbors))
            except ZeroDivisionError:
                pass

            # Update node value
            nx.set_node_attributes(self.G, {i: {'value': new_value}})


    def logistic_map(self, x):
        """
        @x: numeric value (int or float)

        Computes the logistic map f(x) = 1 - a * x^2.
        """

        # Return computed value
        return 1 - self.a * (x**2)


    def pivot(self):
        """
        Rewires a connection from a randomly chosen node to a node with the
        closest value.
        """

        # Pick random pivot node
        pivot = np.random.randint(self.n_v)

        # Get list of neighbors
        neighbors = np.asarray(list(nx.all_neighbors(self.G, pivot)))

        # Return if no neighbors are available
        if len(neighbors) == 0:
            return pivot, pivot, pivot

        # Collect all values
        values = np.asarray(list(nx.get_node_attributes(self.G, 'value').values()))

        # Save pivot value and set to NaN
        pivot_val = values[pivot]
        values[pivot] = np.nan

        # Find candidate
        candidate = np.nanargmin(np.abs(values - pivot_val))

        # Compute outcast
        neighbors_values = values[neighbors]
        outcast = neighbors[np.argmax(np.abs(neighbors_values - pivot_val))]

        # Return pivot and candidate
        return pivot, candidate, outcast

    def rewire(self, pivot, candidate, outcast):
        """
        @pivot: node used as pivot (int)
        @candidate: node to rewire to from pivot (int)
        @outcast: node to drop edge from if needed (int)

        Rewires one of the pivot's connections to the candidate if not
        already connected. If pivot and candidate are the same node, no
        rewire is possible
        """

        # Retrieve all edges connected to pivot node
        edges = list(self.G.edges([pivot]))

        # Check if pivot and candidate are connected and if rewire is possible
        if ((pivot, candidate) not in edges) \
            and (pivot != candidate) \
            and (candidate != outcast):

                # Connect if not yet connected
                self.G.add_edge(pivot, candidate)

                # Drop edge from pivot to outcast
                self.G.remove_edge(pivot, outcast)

    def draw(self):
        """
        Draws the graph.
        """

        # plt.figure(figsize=(8, 8))
        nx.draw_kamada_kawai(self.G, node_color=list(nx.get_node_attributes(self.G, 'value')),
                    cmap=plt.cm.Reds_r, node_size=50)
        # plt.axis('off')
        # plt.show()

In [4]:
def network_plot_3D(G, save=False):

    # Get node positions
    pos = nx.get_node_attributes(G, 'pos')
    
    # Get number of nodes
    n = G.number_of_nodes()

    # Get the maximum number of edges adjacent to a single node
    attr_max = max([G.node[i]['value'] for i in range(n)])

    # Define color range proportional to number of edges adjacent to a single node
    colors = [plt.cm.plasma(G.node[i]['value']/attr_max) for i in range(n)] 

    # 3D network plot
    with plt.style.context(('ggplot')):
        
        fig = plt.figure(figsize=(10,7))
        ax = Axes3D(fig)
        
        # Loop on the pos dictionary to extract the x,y,z coordinates of each node
        for key, value in pos.items():
            xi = value[0]
            yi = value[1]
            zi = value[2]
            
            # Scatter plot
            ax.scatter(xi, yi, zi, c=colors[key], s = 20 , edgecolors='k', alpha=0.7)
        
        # Loop on the list of edges to get the x,y,z, coordinates of the connected nodes
        # Those two points are the extrema of the line to be plotted
        for i,j in enumerate(G.edges()):

            x = np.array((pos[j[0]][0], pos[j[1]][0]))
            y = np.array((pos[j[0]][1], pos[j[1]][1]))
            z = np.array((pos[j[0]][2], pos[j[1]][2]))
        
        # Plot the connecting lines
            ax.plot(x, y, z, c='black', alpha=0.1)
    
    # Set the initial view
    ax.view_init(30,0)

    # Hide the axes
    ax.set_axis_off()

    plt.show()
    
    return


In [6]:
#Animated plot after timesteps

graph = pickle.load(open('graph_10000.p','rb'))

positions = nx.spring_layout(graph.G,dim=3)
pos=dict()
m=0
for key, value in positions.items():
    pos[m]={'pos': value}
    m+=1
nx.set_node_attributes(graph.G,pos)
network_plot_3D(graph.G)

KeyboardInterrupt: 

In [ ]:
#Animated plot before timesteps

graph = Graph(700,8000)

positions = nx.spring_layout(graph.G,dim=3)
pos=dict()
m=0
for key, value in positions.items():
    pos[m]={'pos': value}
    m+=1
nx.set_node_attributes(graph.G,pos)
network_plot_3D(graph.G)